In [4]:
import ollama
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Manip des données et transformation

In [5]:
os.getcwd()

'/home/onyxia/work/Ollama'

In [6]:
df=pd.read_csv('/home/onyxia/work/Ollama/data/1.csv')
df.head()

,Contenu,Lien,Source,Unnamed: 0,annotation_id,annotator,created_at,id,lead_time,sentiment,updated_at
0,Accueil Voyage\nSalon Nautique de Paris 2019: ...,http://www.lefigaro.fr/voyages/salon-nautique-...,LF,0,35,1,2024-07-19T20:42:45.261508Z,1,14.644,autre,2024-07-19T20:42:45.261555Z
1,"Après Commercy et Saint-Nazaire, Montceau-les-...",https://www.humanite.fr/social-et-economie/gil...,MANKIND,1,36,1,2024-07-19T20:42:54.225717Z,2,6.048,autre,2024-07-19T20:42:54.225754Z
2,Les conducteurs de locomotives revendiquent un...,https://www.humanite.fr/monde/allemagne/en-all...,MANKIND,2,37,1,2024-07-19T20:43:04.859624Z,3,7.237,grève,2024-07-19T20:43:04.859681Z
3,Accueil Flash Eco\nLe Covid-19 et ses restrict...,http://www.lefigaro.fr/flash-eco/le-covid-19-e...,LF,7,38,1,2024-07-19T20:43:11.420879Z,4,5.103,autre,2024-07-19T20:43:11.420921Z
4,Saintes : grève des psychologues au centre hos...,https://www.sudouest.fr/charente-maritime/la-r...,SOUTHWEST,8,39,1,2024-07-19T20:43:15.843355Z,5,1.838,grève,2024-07-19T20:43:15.843395Z


In [20]:
df_motif = pd.read_csv('/home/onyxia/work/Ollama/data/motif.csv', encoding='ISO-8859-1',sep=';')

In [77]:
df_motiflast = pd.read_csv('/home/onyxia/work/Ollama/data/motiflast.csv')

In [81]:
df=df_motiflast[['text',"sentiment"]]

In [82]:
df.head()

,text,sentiment
0,« La Croix » absent des kiosques depuis hier L...,Emploi-licenciement
1,Seine-Saint-Denis : les agents techniques des ...,Conditions de travail
2,Trafic perturbé sur le RER E demain Le trafic ...,Conditions de travail
3,Électricité : négociations salariales chez RTE...,Rémunérations
4,EasyJet:appel à la grève Le syndicat Unac des ...,Rémunérations


In [88]:
df["sentiment"].value_counts()

sentiment
5    62
1    33
2    33
4    11
3     2
Name: count, dtype: int64

In [87]:
###Recodage des modalité
recodage = {
    'Autres': 5,
    'Rémunérations': 1,
    'Conditions de travail': 2,
    'Emploi-licenciement': 4,
    'Temps de travail': 3
}

df['sentiment'] = df['sentiment'].replace(recodage)
df.head()

/tmp/ipykernel_1227/1241443443.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace(recodage)
/tmp/ipykernel_1227/1241443443.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace(recodage)


,text,sentiment
0,« La Croix » absent des kiosques depuis hier L...,4
1,Seine-Saint-Denis : les agents techniques des ...,2
2,Trafic perturbé sur le RER E demain Le trafic ...,2
3,Électricité : négociations salariales chez RTE...,1
4,EasyJet:appel à la grève Le syndicat Unac des ...,1


In [7]:
df = df.dropna(subset=['sentiment'])

In [6]:
df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})

/tmp/ipykernel_2787/1150596283.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({"grève": 1, "autre": 0})


In [7]:
df=df[['Contenu','sentiment']]

In [8]:
df['Contenu'] = df['Contenu'].str.replace("\n", " ")

In [89]:
df_train, df_test = train_test_split(df, test_size=0.7, random_state=0)

In [90]:
df_train.head()

,text,sentiment
57,Grève des aiguilleurs SNCF : trafic « très per...,1
120,"Depuis le 7 mars, des salariés d’un magasin pa...",1
65,Limoges : comme un caillou dans la chaussure d...,2
32,Air France prévoit de maintenir 75 % des vols ...,1
129,Basse-Normandie/trains: trafic perturbé Le tr...,4


## Experimentation chat

In [112]:
content=df_train["text"][120]

In [113]:
content

'Depuis le 7 mars, des salariés d’un magasin parisien de l’enseigne d’articles de sport organisent des piquets de grève. Au départ contre la réforme des retraites, leur combat s’est étendu aux questions salariales et de pénibilité.'

In [41]:
message=[
    {
      "role": "system",
      "content": "Je veux t'envoyer un article et tu vas faire une classification binaire. Je veux que tu dises, Oui ou non selon qu il concernent une greve ou pas. Je veux que la sortie sois OUI ou NON"
    },
    {
      "role": "user",
      "content": content
    }
]

In [20]:
message=[
    {
      "role": "user",
      "content": "Certaine ville comportent le mot greve, je veux que tu sois concentré et que tu ne te laisse pas tromper"
    }
]

In [42]:
response = ollama.chat(model='llama3.1', messages=message
)
print(response['message']['content'])

NON


In [140]:
instruction_1="""Je veux t'envoyer un article et tu vas faire une classification binaire. Je veux que tu dises, Oui ou non selon qu il concernent une greve ou pas. Je veux que la sortie sois OUI ou NON
      La greve doit etre considérée dans ce sens : La grève est la dénomination donnée à un mouvement collectif pris à l'initiative de tout ou partie du personnel d'une entreprise, destiné en général à contraindre 
      l'employeur à la négociation des conditions de travail et de rémunération. La grève a cependant changé d'orientation et de modalité car, outre la protestation dont elle se veut être l'expression, elle peut aussi 
      être décidée dans un but de solidarité avec d'autres entreprises même si les protestataires n'appartiennent pas au même employeur, pour exprimer la crainte d'une décision future ou dans un but politique. 
      Elle est exercée tant par le personnel des entreprises privées que par les agents des services publics.
      En cas de doute je veux que tu repondes OUI. Pareil si tu n'a pas de moyen pour classifier la requete que je t'envoie, reponds simplement OUI. 
      Si l'article concerne un preavis de Greve je veux que tu le considere aussi comme une greve, donc tu repond OUI. Ne classe pas les articles qui concernent les marchent ou manifestations politiques
      comme des greves
      """

In [141]:
instruction_2="""
L'aeticle concerne t il la France de l'une des facon suivante:
Est ce que les evenements ont lieux en France ? ou concernent ils une entreprise francaise ou les DOM/TOM  ?  si une des affirmation est 
vraie, reponse OUI, sinon tu reponds NON
"""

In [142]:
instruction_3="""
Si l'article concerne une greve, donne moi le nom de l'entreprise concernée par la greve.
"""

In [143]:
instruction_4="""
Donne ensuite la raison de la grève, n'ajoute pas d'autre details, je veux la raison principale.
"""

In [150]:
instruction="""Je veux t'envoyer un article et tu vas faire une classification binaire. Je veux que tu dises, Oui ou non selon qu il concernent une greve ou pas. Je veux que la sortie sois OUI ou NON
      La greve doit etre considérée dans ce sens : La grève est la dénomination donnée à un mouvement collectif pris à l'initiative de tout ou partie du personnel d'une entreprise, destiné en général à contraindre 
      l'employeur à la négociation des conditions de travail et de rémunération.
      Mon objectif est d'avoir le moins de faux negatifs possible. Etant donné qu'il s'agit d'une classification binaire, je veux pas d'autre reponse que OUI ou NON
      """

In [115]:
instruction="""Je vais t envoyer du texte et il concerne les raison de greve. Je veux que tu le classifie selon 5 modalités:
repond juste 1 s il concerne	 les Rémunérations, si on parle de salaire ou de remuneration dans le texte (cela concerne tout ce qui est lié au salaire)
repond juste 2 s il concerne le	Temps de travail (le texte parle de revendication qui concerne le temps élévé de travail)
repond juste 3 s il concerne	Conditions de travail ( les condition de travail ne sont pas idéale, demande d'amelioration)
repond juste  4 s il concerne	Emploi-licenciement (perte de travail)
repond juste 5 s il concerne	pas tooutes les autres conditions

Je veux que la sortie soit uniquement une des valeurs de 1 à 5 et rien d autre
"""

In [116]:
def ReturnResponse(request):


    message=[ 
        {
      "role": "system",
      "content": instruction
    }
   ,
    {
      "role": "user",
      "content": request
    }
    ]

    response = ollama.chat(model='llama3.1', messages=message
        )
    #print(response['message']['content'])
    #response_1=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la une greve  et reponds juste par oui ou non; l article doit etre considéré comme greve s il s'agit d'arret de travail, de preavis , d intention de greve ou tout autre chose qui se rapporte à un arrete de travail")
    #response_2=ollama.generate(model='llama3.1', prompt="dis moi si " + request + "concene la France ou s il a lieu en France et repond juste par oui ou non Jexige obligatoirement une reponse, si tu douute reponds par oui"  )

    return response['message']['content']

In [144]:
def ReturnResponse(request):
    instructions = [instruction_1,instruction_2,instruction_3,instruction_4]
    
    responses = []

    for instruction in instructions:

        message = [ 
            {
                "role": "system",
                "content": instruction
            },
            {
                "role": "user",
                "content": request
            }
        ]

        response = ollama.chat(model='llama3.1', messages=message)
        
        responses.append(response['message']['content'])
    
    return responses

In [117]:
ReturnResponse(content)

'1'

In [114]:
df_train[['Greve',"Raison_sociale","Revendication"]] = df_train['Contenu'].apply(lambda x: pd.Series(ReturnResponse(x)))

In [118]:
df_train[["Revendication"]] = df_train['text'].apply(lambda x: pd.Series(ReturnResponse(x)))

In [119]:
df_train.head()

,text,sentiment,Revendication
57,Grève des aiguilleurs SNCF : trafic « très per...,1,4
120,"Depuis le 7 mars, des salariés d’un magasin pa...",1,1
65,Limoges : comme un caillou dans la chaussure d...,2,3
32,Air France prévoit de maintenir 75 % des vols ...,1,2
129,Basse-Normandie/trains: trafic perturbé Le tr...,4,4


In [146]:
df_train.iloc[2]['Revendication']

'La raison principale de la grève n\'est pas mentionnée, mais il est fait référence à une "grève" sans plus de détails sur son objet.'

In [68]:
df_train["Catégorie"].value_counts()

Catégorie
5    84
1    51
3    25
4    20
2     5
Name: count, dtype: int64

In [120]:
df_train["Revendication"].value_counts()

Revendication
1    10
5    10
4     8
3     7
2     7
Name: count, dtype: int64

In [120]:
df_train["Greve"].value_counts()

Greve
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [128]:
df_train['Greve']=df_train['Greve'].replace({"NON.": 0,"OUI.": 1, "NON": 0,'OUI':1})

In [129]:
dff=df_train[(df_train.Greve ==1) | (df_train.Greve == 0)]

In [130]:
dff

,Contenu,sentiment,Greve,Raison_sociale,Revendication
82,La violence des déchets Un préavis de grève a ...,1,1,L'entreprise concernée par la grève est les dé...,La raison principale de la grève est : La viol...
189,Accueil Flash Eco Aérien: NKM salue la fin de ...,1,1,Aéroports de Paris (ADP),La raison principale de la grève était la créa...
419,Population de Saint-Quay-Perros : nombre d'hab...,0,0,"L'article concerne Saint-Quay-Perros, et donc ...",La raison principale de la grève n'est pas men...
522,Agression de Samara : « On la fait passer pour...,0,1,L'entreprise concernée par la grève est le col...,« On la fait passer pour la méchante »
280,"Après les Assises du ferroviaire, les syndicat...",1,1,SNCF.,La modification de la réglementation du travail.
...,...,...,...,...,...
75,"L’interview politique, une institution en dang...",0,1,L'article concerne une grève contre les ordonn...,L'opposition atomisée par les élections.
282,Accueil Politique Législative à Mayotte : duel...,0,0,L'entreprise concernée par cette grève général...,La grève générale.
9,Accueil Flash Actu Nigeria/Attentats: évasion ...,0,1,Ni Laipejia Airlines,Les étudiants nigérians sont en grève contre l...
366,Beauvais : le mouvement des conducteurs de bus...,1,1,Beauvaisis Mobilité - Corolis est l'entreprise...,La grève est due au conflit entre la CGT et la...


In [133]:
df1=dff[['Greve']]
df1.rename(columns={'Greve': 'valeur'}, inplace=True)

/tmp/ipykernel_2787/320855277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'Greve': 'valeur'}, inplace=True)


In [134]:
df2=dff[['sentiment']]
df2.rename(columns={'sentiment': 'valeur'}, inplace=True)


/tmp/ipykernel_2787/2142598591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'sentiment': 'valeur'}, inplace=True)


In [126]:
#df2 = df2['valeur']
df2 = df_train['Revendication'].astype(int)

In [127]:
#df1['valeur'] = df1['valeur'].astype(int)
df1 = df_train['sentiment'].astype(int)

## Evaluation

In [128]:
print(confusion_matrix(df2, df1))
print(classification_report(df2, df1))

[[7 0 0 0 3]
 [1 5 0 0 1]
 [1 2 1 1 2]
 [1 1 0 3 3]
 [1 4 0 0 5]]
              precision    recall  f1-score   support

           1       0.64      0.70      0.67        10
           2       0.42      0.71      0.53         7
           3       1.00      0.14      0.25         7
           4       0.75      0.38      0.50         8
           5       0.36      0.50      0.42        10

    accuracy                           0.50        42
   macro avg       0.63      0.49      0.47        42
weighted avg       0.62      0.50      0.48        42



## Un echantillon plus petit

In [ ]:
print(confusion_matrix(df2, df1))
print(classification_report(df2, df1))